In [1]:
from pathlib import Path

folder = Path("data/xlsx")

xlsx_files = [
    str(file) for file in
    folder.glob("*.xlsx")
]

print(xlsx_files)


['data\\xlsx\\2024_C6H6_1g.xlsx', 'data\\xlsx\\2024_CO_1g.xlsx', 'data\\xlsx\\2024_NO2_1g.xlsx', 'data\\xlsx\\2024_O3_1g.xlsx', 'data\\xlsx\\2024_PM10_1g.xlsx', 'data\\xlsx\\2024_PM25_1g.xlsx', 'data\\xlsx\\2024_SO2_1g.xlsx']


In [2]:
import pandas as pd

csv_files = []
for file_path in xlsx_files:
    csv_path = file_path.replace("xlsx", "csv")
    csv_files.append(csv_path)

    df = pd.read_excel(file_path)
    df.to_csv(csv_path, index=False)


In [3]:
import pandas as pd

THE_CODE = "WmOlsPuszkin"


def get_column_from_file(csv_path: str) -> pd.DataFrame:
    name = Path(csv_path).stem.split("_")[1]

    df = pd.read_csv(csv_path, low_memory=False)

    columns_to_keep = [df.columns[0]]

    for col in df.columns[1:]:
        if df[col].iloc[0] == THE_CODE:
            columns_to_keep.append(col)
            break

    df = df[columns_to_keep]

    df = df.iloc[5:].reset_index(drop=True)

    df.columns = ["time", name]

    return df, name


dfs = []
names = []
for csv_path in csv_files:
    df, name = get_column_from_file(csv_path)
    names.append(name)
    dfs.append(df)

df = dfs[0]

for the_df in dfs[1:]:
    df = df.merge(the_df, on="time", how="inner")

df["time"] = pd.to_datetime(df["time"], errors="coerce")
for col in names:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.to_csv("data/combined_data.csv", index=False)
print(df.head())


                 time  C6H6     CO  NO2    O3  PM10  PM25   SO2
0 2024-01-01 01:00:00  0.63  0.342  8.6  42.3  39.2  37.4  11.3
1 2024-01-01 02:00:00  0.52  0.342  5.3  43.8  19.6  18.8   9.5
2 2024-01-01 03:00:00  0.57  0.351  6.3  41.4  19.8  19.0   9.5
3 2024-01-01 04:00:00  0.58  0.352  5.6  40.9  19.4  18.7   9.7
4 2024-01-01 05:00:00  0.61  0.368  5.9  39.7  19.8  19.0  10.1
